In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_vertical_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,74075.41,-0.46,-0.03,0.31,0.01,-0.04,0.00,-0.00,-0.00
3,-0.46,18610.86,-0.11,0.05,-0.05,0.01,0.02,0.00,0.00
4,-0.03,-0.11,4603.62,0.08,0.00,0.00,-0.01,-0.00,0.00
5,0.31,0.05,0.08,1031.75,0.00,-0.00,0.00,0.00,0.00
6,0.01,-0.05,0.00,0.00,210.16,-0.00,-0.00,0.00,0.00
7,-0.04,0.01,0.00,-0.00,-0.00,39.20,0.00,0.00,0.00
8,0.00,0.02,-0.01,0.00,-0.00,0.00,6.62,0.00,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,1.01,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.12


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00003,0.00000,-0.00002,0.00001,-0.00001,-0.00000
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00001,0.00004,0.00001,0.00000
4,-0.00000,-0.00001,1.00000,0.00004,0.00000,0.00000,-0.00003,-0.00000,0.00001
5,0.00003,0.00001,0.00004,1.00000,0.00000,-0.00002,0.00005,0.00000,0.00001
6,0.00000,-0.00003,0.00000,0.00000,1.00000,-0.00001,-0.00000,0.00002,0.00000
7,-0.00002,0.00001,0.00000,-0.00002,-0.00001,1.00000,0.00002,0.00000,0.00000
8,0.00001,0.00004,-0.00003,0.00005,-0.00000,0.00002,1.00000,0.00003,0.00000
9,-0.00001,0.00001,-0.00000,0.00000,0.00002,0.00000,0.00003,1.00000,-0.00000
10,-0.00000,0.00000,0.00001,0.00001,0.00000,0.00000,0.00000,-0.00000,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.8156914259557112

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.10365071e+08 3.78972827e+07 2.27101887e+06 1.10916038e+05
 4.41901402e+03 1.44208053e+02 3.66005500e+00 6.41360072e-02
 8.19909752e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999409,-0.033453,-0.007720,-0.001719,-0.000351,-0.000066,-0.000011,-0.000002,-1.980738e-07
1,0.033123,0.998702,-0.037843,-0.007912,-0.001609,-0.000299,-0.000050,-0.000008,-8.863112e-07
2,0.008887,0.037213,0.998455,-0.039555,-0.007608,-0.001415,-0.000240,-0.000037,-4.199167e-06
3,0.002311,0.009232,0.038846,0.998256,-0.042729,-0.007591,-0.001275,-0.000195,-2.258644e-05
4,0.000565,0.002252,0.009106,0.041957,0.997865,-0.048526,-0.007846,-0.001194,-1.386993e-04
5,0.000132,0.000523,0.002116,0.009456,0.047603,0.997092,-0.058083,-0.008529,-9.897400e-04
6,0.000029,0.000116,0.000475,0.002113,0.010399,0.056782,0.995243,-0.077960,-8.746803e-03
7,0.000007,0.000027,0.000108,0.000480,0.002358,0.012653,0.075796,0.990220,-1.164180e-01
8,0.000002,0.000006,0.000025,0.000113,0.000554,0.002970,0.017591,0.115378,9.931613e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0005910633354971884,
 0.001298090919549244,
 0.00154534790736216,
 0.0017438417538505613,
 0.0021354992663350503,
 0.0029080124155455023,
 0.0047569700747494,
 0.00977984825927447,
 0.006838716460861205]